In [25]:
from scipy.spatial import distance
import winsound
from imutils import face_utils
import imutils
import dlib
import cv2
from pyicloud import PyiCloudService
import rabbitmqDevice
import getpass

In [26]:
email = input("Enter icloud email:")
password = getpass.getpass(prompt = "Enter icloud password:")
api = PyiCloudService(email, password)
if api.requires_2fa:
    print("Two-factor authentication required.")
    code = input("Enter the code you received of one of your approved devices: ")
    result = api.validate_2fa_code(code)
    print("Code validation result: %s" % result)

    if not result:
        print("Failed to verify security code")
        sys.exit(1)

    if not api.is_trusted_session:
        print("Session is not trusted. Requesting trust...")
        result = api.trust_session()
        print("Session trust result %s" % result)

        if not result:
            print("Failed to request trust. You will likely be prompted for the code again in the coming weeks")
elif api.requires_2sa:
    import click
    print("Two-step authentication required. Your trusted devices are:")

    devices = api.trusted_devices
    for i, device in enumerate(devices):
        print(
            "  %s: %s" % (i, device.get('deviceName',
            "SMS to %s" % device.get('phoneNumber')))
        )

    device = click.prompt('Which device would you like to use?', default=0)
    device = devices[device]
    if not api.send_verification_code(device):
        print("Failed to send verification code")
        sys.exit(1)

    code = click.prompt('Please enter validation code')
    if not api.validate_verification_code(device, code):
        print("Failed to verify verification code")
        sys.exit(1)

device=api.devices[0]

In [27]:
def eye_aspect_ratio(eye):
	A = distance.euclidean(eye[1], eye[5])
	B = distance.euclidean(eye[2], eye[4])
	C = distance.euclidean(eye[0], eye[3])
	ear = (A + B) / (2.0 * C)
	return ear

In [28]:
thresh = 0.25
frame_check = 20
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [29]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]


In [30]:
cap=cv2.VideoCapture(0)
flag=0
while True:
        ret, frame=cap.read()
        frame = imutils.resize(frame, width=450)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        subjects = detect(gray, 0)
        for subject in subjects:
            shape = predict(gray, subject)
            shape = face_utils.shape_to_np(shape)#converting to NumPy Array
            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
            if ear < thresh:
                flag += 1
                #print (flag)
                if flag >= frame_check:
                    #winsound.Beep(440, 3000)
                    cv2.putText(frame,str(flag), (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

                    cv2.putText(frame, "****************ALERT!****************", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                        
                    cv2.putText(frame, "****************ALERT!****************", (10,325),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    rabbitmqDevice.sendNotif(device)
            else:
                flag = 0
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            cv2.destroyAllWindows()
            cap.release()
            break

{'isOld': False, 'isInaccurate': False, 'altitude': 0.0, 'positionType': 'Unknown', 'secureLocation': None, 'secureLocationTs': 0, 'latitude': 36.850093848946514, 'floorLevel': 0, 'horizontalAccuracy': 10.549633306018501, 'locationType': '', 'timeStamp': 1654037951492, 'locationFinished': False, 'verticalAccuracy': 0.0, 'locationMode': None, 'longitude': 10.259924906948125}
36.850093848946514
 [x] Sent 
{'isOld': False, 'isInaccurate': False, 'altitude': 0.0, 'positionType': 'Unknown', 'secureLocation': None, 'secureLocationTs': 0, 'latitude': 36.850093848946514, 'floorLevel': 0, 'horizontalAccuracy': 10.549633306018501, 'locationType': '', 'timeStamp': 1654037951492, 'locationFinished': False, 'verticalAccuracy': 0.0, 'locationMode': None, 'longitude': 10.259924906948125}
36.850093848946514
 [x] Sent 
{'isOld': False, 'isInaccurate': False, 'altitude': 0.0, 'positionType': 'Unknown', 'secureLocation': None, 'secureLocationTs': 0, 'latitude': 36.850093848946514, 'floorLevel': 0, 'horiz

In [ ]:
cv2.destroyAllWindows()
cap.release()